In [ ]:
import tensorflow as tf
from   tensorflow import keras
from   tensorflow.keras import regularizers
from   tensorflow.keras import Sequential
from   tensorflow.keras.layers import Dropout, Dense

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [ ]:
from   IPython import display
from   matplotlib import pyplot as plt

In [ ]:
import errno
import glob
import json
import numpy as np
import os
import pandas as pd
import pathlib
import shutil
import subprocess
import tempfile
import uuid

In [ ]:
# Local modules
import config
import utils

In [ ]:
# Globals
TICKER = 'SPY'

In [ ]:
# For saving the model
PREFIX = 'model'
TICKER_MODEL_DIR = os.path.join(config.ML_MODELS_DIR, TICKER)

In [ ]:
# Set the values to be used for working with the data
BATCH_SIZE = 512
BUFFER_SIZE = 100
MAX_MARGIN = 5
MIN_PROFIT = 1 + utils.calculate_fee()
DATA_SPLIT = 0.90
MAX_EPOCHS = 200

In [ ]:
# For tensorboard
LOGDIR = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(LOGDIR, ignore_errors=True)

In [ ]:
try:
    os.makedirs(TICKER_MODEL_DIR)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
    for tmpdir in glob.glob('{}/{}*'.format(TICKER_MODEL_DIR, PREFIX)):
        shutil.rmtree(tmpdir)
MODEL_DIR = tempfile.mkdtemp(prefix='model', dir=TICKER_MODEL_DIR)

In [ ]:
# Load the data
examples_df = pd.read_pickle(
    os.path.join(
        config.ML_DATA_DIR,
        TICKER,
        'examples',
        '1e881bf3-0a2b-429e-af1e-5760c59cc41c'
    )
)

In [ ]:
# Collect the statistics
pooled_means, pooled_variances = utils.pool_stats_from_stats_df(TICKER)

In [ ]:
# Immediately save the metadata that we can
pooled_means.to_pickle(os.path.join(MODEL_DIR, 'means'))
pooled_variances.to_pickle(os.path.join(MODEL_DIR, 'variances'))
pooled_stds = pooled_variances.pow(1/2)

In [ ]:
pooled_means

In [ ]:
# We don't need the description or open_margin anymore
examples_df.drop(['description'], axis=1, inplace=True)
examples_df.drop(['open_margin'], axis=1, inplace=True)
# Pop out the max_profit and compare it to our desired minimum profit
labels = examples_df.pop('max_profit') >= MIN_PROFIT

In [ ]:
n_examples, n_features = examples_df.shape
print('{} examples\n{} features'.format(n_examples, n_features))

In [ ]:
feature_order = examples_df.columns.tolist()
print(feature_order)

In [ ]:
# Normalize all of the stuff that will be used for X.
examples_df = (examples_df - pooled_means[examples_df.columns]) / pooled_stds[examples_df.columns]

In [ ]:
# Build the datasets for train and validation
dataset = tf.data.Dataset.from_tensor_slices((examples_df.values, labels.values)).shuffle(n_examples)

# Split up the data
n_train = int(n_examples * DATA_SPLIT)
train_dataset = dataset.take(n_train)
test_dataset = dataset.skip(n_train)

STEPS_PER_EPOCH = n_train//BATCH_SIZE

train_dataset = train_dataset.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).repeat()
validate_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    decay_steps=STEPS_PER_EPOCH*5,
    decay_rate=1,
    staircase=False
)

checkpoint_filepath = os.path.join(MODEL_DIR, 'checkpoint')
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

class MetadataSaver(keras.callbacks.Callback):
    _best_loss = np.inf
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_loss'] >= self._best_loss:
            return
        
        self._best_loss = logs['val_loss']
        with open(os.path.join(MODEL_DIR, 'metadata'), 'w') as MF:
            json.dump(
                {
                    'ticker': TICKER,
                    'max_margin': MAX_MARGIN,
                    'min_profit': MIN_PROFIT,
                    'feature_order': feature_order,
                    'accuracy': float(logs['val_accuracy']),
                    'loss': float(self._best_loss),
                },
                MF
            )

def get_callbacks(name):
    return [
        # tfdocs.modeling.EpochDots(),
        model_checkpoint_callback,
        MetadataSaver(),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
        tf.keras.callbacks.TensorBoard(LOGDIR/name),
    ]

model = Sequential([
    Dense(512, activation='relu', input_shape=(n_features,)),
    Dense(256, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(lr_schedule),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[
        tf.keras.losses.BinaryCrossentropy(
            from_logits=True, name='binary_crossentropy'),
        'accuracy'
    ]
)

model.summary()

In [ ]:
model.fit(train_dataset,
          steps_per_epoch = STEPS_PER_EPOCH,
          epochs=MAX_EPOCHS,
          validation_data=validate_dataset,
          callbacks=get_callbacks('testing'),
          verbose=1)

In [ ]:
# Build a tarball for this session (ignoring the directory when including filenames)
files_to_tar = [os.path.basename(f) for f in glob.glob('{}/*'.format(MODEL_DIR))]
tarball_path = os.path.join(config.ML_MODELS_DIR, TICKER, '{}.tar'.format(uuid.uuid4()))
subprocess.check_call(['tar', '-C', MODEL_DIR, '-cf', tarball_path] + files_to_tar)
shutil.rmtree(MODEL_DIR, ignore_errors=True)

In [ ]:
#docs_infra: no_execute

# Load the TensorBoard notebook extension
%load_ext tensorboard

# Open an embedded TensorBoard viewer
%tensorboard --logdir {LOGDIR}/testing